# Create an API Function


In [13]:
from requests import post as rpost

def call_llama(prompt):
    headers = {"Content-Type": "application/json"}
    payload = {
        "model": "llama3.1",
        "prompt": prompt,
        "stream": False,
    }

    response = rpost(
        "http://localhost:11434/api/generate",
        headers=headers,
        json=payload
    )
    return response.json()["response"]

# or use ollama python library 

In [12]:
call_llama(prompt="Why is the skye blue?")

'The color of the sky can vary depending on several factors, but in general, it\'s not actually blue. Here are a few reasons why we might perceive the sky as blue:\n\n1. **Atmospheric scattering**: When sunlight enters Earth\'s atmosphere, it encounters tiny molecules of gases such as nitrogen and oxygen. These molecules scatter the shorter (blue) wavelengths of light more than the longer (red) wavelengths, giving the sky its blue appearance.\n2. **Refraction**: As light passes through the atmosphere, it is refracted, or bent, which can also contribute to the sky\'s color. However, this effect is relatively small compared to scattering.\n3. **Perception**: Our eyes and brains play a significant role in how we perceive the color of the sky. The blue color we see is an interpretation of the scattered light by our visual system.\n\nNow, why might the sky appear more vividly blue on certain days?\n\n1. **Cleaner atmosphere**: When there are fewer particles in the air (e.g., after a rain sh

# Create a LangChain LLM


In [14]:
from langchain_core.language_models.llms import LLM

class LLaMa(LLM):
    def _call(self, prompt, **kwargs):
        return call_llama(prompt)

    @property
    def _llm_type(self):
        return "llama-3.1-8b"

# Integrating the RAG Agent


In [ ]:
!pip install langchain-huggingface -q 
!pip install faiss-cpu -q 

In [5]:
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

documents = [
    {"content": "What is your return policy? ..."},
    {"content": "How long does shipping take? ..."},
    # Add more documents as needed
]

texts = [doc["content"] for doc in documents]

retriever = FAISS.from_texts(
    texts,
    HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
).as_retriever(k=5)

# Create the Prompt Template

In [6]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

faq_template = """
You are a chat agent for my E-Commerce Company. As a chat agent, it is your duty to help the human with their inquiry and make them a happy customer.

Help them, using the following context:
<context>
{context}
</context>
"""

faq_prompt = ChatPromptTemplate.from_messages([
    ("system", faq_template),
    MessagesPlaceholder("messages")
])

# Create Document and Retriever Chains


In [9]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

document_chain = create_stuff_documents_chain(LLaMa(), faq_prompt)

def parse_retriever_input(params):
    return params["messages"][-1].content

retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever
).assign(answer=document_chain)

# Prompt Your RAG Agent


In [10]:
from langchain.schema import HumanMessage

response = retrieval_chain.invoke({
    "messages": [
        HumanMessage("I received a damaged item. I want my money back.")
    ]
})

print(response)

{'messages': [HumanMessage(content='I received a damaged item. I want my money back.')], 'context': [Document(page_content='What is your return policy? ...'), Document(page_content='How long does shipping take? ...')], 'answer': "I'm so sorry to hear that you received a damaged item!\n\nFirst, let me help you with getting a refund. According to our return policy, if an item arrives damaged or is not as described, we will gladly offer a full refund or replacement.\n\nTo initiate the return process, I just need some information from you. Could you please confirm your order number and provide more details about the damage? This will help us expedite the process and get your refund processed quickly.\n\nOnce we receive the damaged item back (if possible) or verify the issue, we'll be happy to provide a full refund to the original payment method.\n\nIs there anything else I can assist you with regarding this return?"}
